In [241]:
from celery import Celery
import os
from dotenv import load_dotenv
from ollama import Client
import httpx
from langchain_community.chat_models.ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

In [242]:
client = Client(host="http://localhost:11434/api/generate")

In [243]:
# markdown 파일 생성
def save_md_to_file(markdown_str : str, directory : str, file_name : str) :
    # 디렉토리가 존재하지 않으면 생성
    if not os.path.exists(directory):
        os.makedirs(directory)
        
     # 파일 경로 생성
    file_path = os.path.join(directory, file_name)
    
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(markdown_str)
    
    # 파일 객체 정보 변환
    file_info = {
        'file_path' : file_path,
        'file_name' : os.path.getsize(file_path)
    }
    
    return file_info

In [244]:
# 각 파일당 하나씩 프롬프트 생성하기
# You are an experienced technical writer skilled in documenting and formatting code reviews. 
# Your task is to take the results from a code review and format them into a markdown file. 
# The markdown file must be written in English and should include the following sections: Analysis Summary, Key Features, Pre-condition Check, Runtime Error Check, Optimization, Security Issue, and Evaluation. 
# Each section should be formatted appropriately:

# - **Analysis Summary:** Summarize the code review findings in one or two lines.
# - **Key Features:** Analyze and describe the key features of the added or modified files.
# - **Precondition checks:** Checks that a function or method has the necessary variable states or ranges of values to function correctly.
# - **Runtime error checking:** Examines code for possible runtime errors and identifies other potential risks.
# - **Optimization:** Scan your code and recommend optimized code. When recommending code, be sure to include the full source of the file. Please write your code using code blocks to conform to the markdown format - this is a must. 
# - **Security issues:** Scans your code to see if it uses modules with serious security flaws or contains security vulnerabilities.
# - **Evaluation:** Comprehensively evaluates your work. Consider the quality, functionality, and maintainability of the code.
            
# Ensure the markdown document is clear, well-structured, and easy to read.
user_prompt = '''
Your task is to take the results from a code review and format them into a markdown file. 
The markdown file must be written in Korean and should include the following sections: 
- **Analysis Summary:** Summarize the code review findings in one or two lines.
- **Key Features:** Analyze and describe the key features of the added or modified files.
- **Precondition checks:** Checks that a function or method has the necessary variable states or ranges of values to function correctly.
- **Runtime error checking:** Examines code for possible runtime errors and identifies other potential risks.
- **Optimization:** Scan your code and recommend optimized code. When recommending code, be sure to include the full source of the file. Please write your code using code blocks to conform to the markdown format - this is a must. 
- **Security issues:** Scans your code to see if it uses modules with serious security flaws or contains security vulnerabilities.
- **Evaluation:** Comprehensively evaluates your work. Consider the quality, functionality, and maintainability of the code.

When writing a markdown file, make sure to write a subheading, and make sure to write it according to the format so that the next line contains the appropriate subheading.
Remember to write the markdown file in Korean.

```python
# gitlab_hook.py
from fastapi import APIRouter, HTTPException, Path, Request
from fastapi.responses import JSONResponse
from models import WebHookResponse, ErrorResponse, PromptCreateRequest
from dotenv import load_dotenv
import os
import git
from urllib.parse import urlparse
from celery_app import llm_code_review_task, llm_code_review_task_by_langchain
from services.create_prompt import create_prompt
import shutil

# .env 파일에서 환경 변수 로드
load_dotenv()

router = APIRouter()

# 로컬 리포지토리 경로 설정
LOCAL_REPO_PATH = os.getenv('LOCAL_REPO_PATH', "/tmp/repo")
# 퍼스널 액세스 토큰 또는 SSH 키 사용 시 필요
ACCESS_TOKEN = os.getenv('ACCESS_TOKEN', "") 
# gitlab reverse proxy
PROEJCT_CONTEXT_PATH = os.getenv('PROEJCT_CONTEXT_PATH', "")

@router.post("/git/webhook")
async def gitlab_webhook(request : Request):        
    payload = await request.json()
    print(payload)
    
    # 필요한 정보를 추출
    project_info = payload['project']
    project_url= project_info['http_url']
    # URL 파싱
    parsed_url = urlparse(project_url)
    # 경로 추출
    extracted_part = parsed_url.path
    repo_url = f"http://oauth2:{ACCESS_TOKEN}@{PROEJCT_CONTEXT_PATH}{extracted_part}"
    
    commit_sha = payload['checkout_sha']
    commits = payload['commits']
    
    # 각각 커밋 제목, 내용, 추가, 수정 파일 목록을 저장할 딕셔너리    
    commit_id_list = []
    title_dict = {}
    message_dict = {}    
    added_files_content = {}
    modified_files_content = {}    
    
    try :                
        # 로컬 디렉토리 정리
        if os.path.exists(LOCAL_REPO_PATH):
            shutil.rmtree(LOCAL_REPO_PATH)
        # 리포지토리 클론
        repo = git.Repo.clone_from(repo_url, LOCAL_REPO_PATH)
        # 특정 커밋으로 체크아웃
        repo.git.checkout(commit_sha)
            
        # ## FIXME 이부분 지금 오류남
        # if not os.path.exists(LOCAL_REPO_PATH) :
        #     # 레포지토리 정보가 없다면 클론
        #     repo = git.Repo.clone_from(repo_url, LOCAL_REPO_PATH)
        # else :
        #     # 이미 존재하는 경우 변경 사항만 가져옴
        #     repo = git.Repo(LOCAL_REPO_PATH)
        #     origin = repo.remotes.origin
        #     origin.pull()
                    
        # # 특정 커밋으로 체크아웃
        # repo.git.checkout(commit_sha)
        
        # 수정된 파일 목록 가져오기 및 파일 내용 읽기
        for commit in commits:            
            commit_id = commit["id"]    # commit id
            commit_title = commit["title"]  # title
            commit_message = commit["message"] # message
            
            # 제목과 타이틀 저장            
            commit_id_list.append(commit_id)
            title_dict[commit_id] = commit_title
            message_dict[commit_id] = commit_message          
            
            # 추가된 파일
            for file_path in commit["added"]:
                absolute_file_path = os.path.join(LOCAL_REPO_PATH, file_path)
                
                # 파일 디렉토리가 존재하는지 확인하고 파일 내용 읽기
                if os.path.exists(absolute_file_path):
                    with open(absolute_file_path, 'r') as file:
                        added_files_content[file_path] = file.read()                                                        
            # 수정된 파일
            for file_path in commit["modified"]:
                absolute_file_path = os.path.join(LOCAL_REPO_PATH, file_path)

                # 파일 디렉토리가 존재하는지 확인하고 파일 내용 읽기
                if os.path.exists(absolute_file_path):
                    with open(absolute_file_path, 'r') as file:
                        modified_files_content[file_path] = file.read()                                        
    except Exception as e:
        response = ErrorResponse(detail=str(e)) 
        return JSONResponse(status_code=400, content=response.model_dump())        
    
    # prompt 생성
    request = PromptCreateRequest(id_list=commit_id_list, title_dict=title_dict, message_dict=message_dict, added_files_content=added_files_content, modified_files_content=modified_files_content)
    user_prompt = create_prompt(request)        
    print("====")
    print(user_prompt)    
    print("====")
    task = llm_code_review_task.apply_async(args=[user_prompt])    
    # task2 = llm_code_review_task_by_langchain.apply_async(args=[user_prompt])    
    # llm_code_review_task
    # llm_code_review_task_by_langchain
    print("====")
    print(task.id)    
    print("====")
    return JSONResponse(status_code=200, content={"detail" : task.id, "detail2" : ""})

```
'''

In [245]:
response = client.chat(
    # model='llama3.1', 
    model='mistral-nemo', 
    options={
        'temperature': 0.1,
    },
    messages=[
        {
            'role' : 'system',
            'content' : """
            You are an expert software analyst. Users will provide you with the entire committed source code for review. 
            Your task is to analyze the code, present your findings, and offer an optimized version if possible.

            As you analyze the source code, consider the following topics:
            - **Analysis Summary**: Provide a brief summary of your code review findings in one or two lines.
            - **Key Features**: Describe the key features of each file or component in the codebase.
            - **Prerequisite Checks**: Ensure that the functions or methods have the necessary variable states or value ranges to operate correctly.
            - **Runtime Error Checking**: Identify possible runtime errors and other potential risks within the code.
            - **Optimization**: Suggest performance improvements, identifying code sections that could be optimized.
            - **Security Issues**: Check for the use of modules with known security flaws or any potential security vulnerabilities in the code.
            - **Evaluation**: Give a comprehensive evaluation of the code's quality, functionality, and maintainability.

            Please format your response in markdown, ensuring it is easy to parse. 

            For optimizations, include the entire optimized code within code blocks in markdown format.
            Do not include any additional text outside the specified response values.
            Always write the entire code in the form of code blocks in Markdown for optimizations.
            Write answer in Korean.
            """
            # 'content' : """
            # You're a very good software analyst. From now on, users will show you the entire committed source code. Take a look at it, analyze it, and tell us what you find. 
            # After looking at the source code, present an optimized version of the code, including performance improvements. 

            # As you analyze the source code, keep the following topics in mind as you do so
            # - Analysis summary: Summarize your code review findings in one or two lines.
            # - Key features: Analyze and describe the key features of each file.
            # - Prerequisite checks: Verify that the function or method has the necessary variable states or value ranges to function correctly.
            # - Runtime error checking: Inspect your code for possible runtime errors and identify other potential risks.
            # - Optimization: Scans code patches for optimization points and recommends optimized code if it appears to be degrading performance. 
            # - Security issues: Scan your code to see if it uses modules with serious security flaws or contains security vulnerabilities.
            # - Evaluation: Evaluate your work comprehensively. Considers the quality, functionality, and maintainability of the code.
            
            # Please make sure to write the results according to the above topic.
            # Return a response in markdown so that your analysis is easy to parse.
            # The topics above are the same as the subheadings in your final analysis. In particular, be sure to write the entire code in the form of code blocks in Markdown for optimizations.            
            # Do not write any additional text other than the response values in Markdown format.
            # """,            
        },
        {
            'role': 'user',            
            'content':  user_prompt ,
        },
    ],
)      

In [246]:
ai_response = response['message']['content']
# \n을 </br>로 변경
# ai_response = ai_response.replace('\n', '</br>')

file_dir = "./"
file_name = "example_origin.md"
file_info = save_md_to_file(ai_response, file_dir, file_name)

In [247]:
# 이제 응답을 한글로만 하면 됨 ... ??
korean_reponse = client.chat(
    model='llama3.1', 
    options={
        'temperature': 0,
    },
    messages=[
        {
            'role' : 'system',
            'content' : """
            You are an excellent translator. Please translate the markdown written by the user into Korean. 
            Don't translate the source code in the code block and leave it as it is in the original text.            
            """,            
        },
        {
            'role': 'user',            
            'content':  response['message']['content'] ,
        },
    ],
)      

In [248]:
file_dir = "./"
file_name = "example_translate.md"
file_info = save_md_to_file(korean_reponse['message']['content'], file_dir, file_name)